In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd
from IPython.display import clear_output

# Code combining into Auth Year combination sets

##### Prelim Data Munge

In [ ]:
#Date to years function - works on application date and publication date in one
def date_to_year(df):
    if 'appln_filing_date' in df.columns:
        df.appln_filing_date = df.appln_filing_date.map(lambda x:int(x[:4]))
        df.rename(columns={'appln_filing_date':'app_year'}, inplace=True)
    if 'publn_date'in df.columns:
        df.publn_date = df.publn_date.map(lambda x:int(x[:4]))
        df.rename(columns={'publn_date':'pub_year'}, inplace=True)
    return

## Importing
[to top](#Table-of-Contents) 

In [ ]:
pwd

In [ ]:
#%time df1 = pk.load(open('pickles_and_munged_csv/final_data_v3.pk','rb'))
%time df2 = pd.read_csv('pickles_and_munged_csv/final_data_codes_as_ints_v1.csv', engine='c', dtype={'publn_nr':str,'appln_id':str})
# pickling is so so slow...

In [ ]:
print df2.appln_auth.unique()
print len(df2.appln_auth.unique())

In [ ]:
# countries appearing before 1950
cntr_pre1950 = pk.load(open('pickles_and_munged_csv/cntr_ls_appear_before_1950.pk','rb'))
cntr_pre1950

In [ ]:
# munge dates to years
date_to_year(df2)

In [ ]:
df2[df2.appln_auth.isin(cntr_pre1950)]

In [ ]:
df2 = df2[df2.appln_auth.isin(cntr_pre1950)]

# Group - Apply - Combine: Auth:Year:ID, 

In [ ]:
# Group
grp_ayi = df2.groupby(['appln_auth', 'pub_year', 'appln_id'])

In [ ]:
# Apply - combine codes for each app into list NOTE time is for 100000 
%time app_id_class_ls =  grp_ayi['ipc_class_symbol'].apply(list); app_id_class_ls.head()

In [ ]:
# regroup by just auth - year
grp_ay = app_id_class_ls.groupby(level=[0,1])

In [ ]:
# Apply 2 - combine lists of codes for each year into list
au_yr_ls_ls = grp_ay.apply(list); au_yr_ls_ls.head()

In [ ]:
yr_lsls_au = au_yr_ls_ls.reset_index(level=0).pivot(columns='appln_auth'); yr_lsls_au
yr_lsls_au.columns = yr_lsls_au.columns.droplevel(); yr_lsls_au.head()

In [ ]:
def map_nan_lsls(ls):
    if type(ls) != type([]):
        ls = []
    return ls

In [ ]:
yr_lsls_au = yr_lsls_au.applymap(map_nan_lsls); yr_lsls_au.head()

In [ ]:
# gotta pickle it, writing to csv doesn't preserve the columns and index properly...
pk.dump(yr_lsls_au, open('pickles_and_munged_csv/pre1950_cated_codes_as_ints_v1.pk', 'wb'))

In [ ]:
yr_lsls_au.applymap(len)

# Above is THE data frame for building networks
NetworkX can take the list of lists and add links for all permuations of the items in each list
The idea being that we go from dataframe with each value being an encoding of the adjaceny matric for each country for that year, then apply a mapping to each value that's a function using networkx to turn that into a graph, thus leaving the same data frame but with nx graphs instead of lists of lists. Then this will become the root dataframe. 
## Calling this type of dataframe a code sets frame

In [ ]:
for col in yr_lsls_au:
    print col
    for inx in yr_lsls_au.index[1:]:
        print inx,
        yr_lsls_au.loc[inx,col] = yr_lsls_au.loc[inx,col] + yr_lsls_au.loc[inx-1,col]
clear_output()

In [ ]:
# gotta pickle it, writing to csv doesn't preserve the columns and index properly...
pk.dump(yr_lsls_au, open('pickles_and_munged_csv/pre1950_cated_cumed_codes_as_ints_v1.pk', 'wb'))

In [ ]:
yr_lsls_au.applymap(len)